In [1]:
#initialise general libraries
import os
import numpy as np
import pandas as pd
from pandas import DataFrame
import pickle
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

from matplotlib import pyplot as plt
%matplotlib inline

# Allows more columns to be shown for outputs
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 5200)

In [2]:
#set directories
dirPData = "C:/Users/kevin/Google Drive/AML/Projects/Coursework/PData/"
dirPOutput = "C:/Users/kevin/Google Drive/AML/Projects/Coursework/POutput/"

In [3]:
#import variables
f_name = dirPData + 'ohprepVars.pickle'

with open(f_name, "rb") as f:
    dict_ = pickle.load(f)
    
df_test = dict_['df_test']
df_train = dict_['df_train']
OH_df_test = dict_['OH_df_test']
OH_df_train = dict_['OH_df_train']
del f_name, dict_

#prepare X and y
train_y = df_train['target'].values

var_notToUse = ['id']

allFeatures = [var for var in OH_df_train]

train_X = OH_df_train[allFeatures]
test_X = OH_df_test[allFeatures]

#prepare validation data
train_Xv, test_Xv, train_yv, test_yv = train_test_split(train_X, train_y, test_size=0.25, random_state=1)

## Histogram-based Gradient Boosting Classification 
### Kaggle Score: 0.87152, Validation Score: 0.8802352181662525
<img src="histgbm3.png">

In [4]:
hgbm_model1v = HistGradientBoostingClassifier(random_state=1, learning_rate=0.111, l2_regularization=0.11, max_iter=39, max_leaf_nodes=25, min_samples_leaf=25)
hgbm_model1v.fit(train_Xv, train_yv)
hgbmprediction1v = hgbm_model1v.predict_proba(test_Xv)
print("Validation score:",roc_auc_score(test_yv, hgbmprediction1v[:,1]))

Validation score: 0.8802352181662525


In [5]:
hgbm_model3 = HistGradientBoostingClassifier(random_state=1, learning_rate=0.111, l2_regularization=0.111, max_iter=39, max_leaf_nodes=25, min_samples_leaf=25)
hgbm_model3.fit(train_X, train_y)
hgbmprediction3 = hgbm_model3.predict_proba(test_X)

In [6]:
hgbmpred3 = {'id': df_test['id'].values,
        'target': hgbmprediction3[:,1]}
df_hgbmpred3 = DataFrame(hgbmpred3)

df_hgbmpred3 = df_hgbmpred3.reset_index().set_index('id')
df_hgbmpred3 = df_hgbmpred3.drop(columns='index')
df_hgbmpred3.head()

,target
id,
5,0.025898
10,0.087922
11,0.026751
13,0.160207
16,0.019530


In [7]:
df_hgbmpred3.to_csv(r'C:/Users/kevin/Google Drive/AML/Projects/Coursework/POutput/histgbm3.csv')

In [8]:
niter = hgbm_model3.n_iter_
ts = hgbm_model3.train_score_
vs = hgbm_model3.validation_score_

print("Model ran", niter, "iterations, had", ts, "train score and", vs, "validation score")

Model ran 39 iterations, had [] train score and [] validation score


* The train and validation scores are both empty which show the model was never stopped early